<a href="https://colab.research.google.com/github/gunobo/bssm-wintercamp_python/blob/main/%EB%B6%80%EC%82%B0%EC%86%8C%EB%A7%88%EA%B3%A02026%EA%B2%A8%EC%9A%B8%EC%BA%A0%ED%94%84(%ED%8C%80%ED%94%84%EB%A1%9C%EC%A0%9D%ED%8A%B8).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **BSSM급식알리미**

이 프로그램은 나이스(NEIS) Open API와 **Jupyter 위젯(ipywidgets)**을 이용해 만든 급식 조회 프로그램입니다.
부산소프트웨어마이스터고의 급식 정보를 날짜별·메뉴별로 조회할 수 있도록 만든 Python 코드입니다.
아쉽게도 colab같은 juypter NoteBook에서는 프로그램 창을 띄우지 못해 HTML을 활용하여 만들었습니다.

### ✅ 주요 기능

📅 날짜별 급식 조회
원하는 날짜를 선택하면 조식 / 중식 / 석식 메뉴를 한 번에 확인할 수 있습니다.

⚠️ 알레르기 정보 표시
메뉴에 포함된 알레르기 번호를 자동 추출 번호 버튼을 클릭하면 해당 알레르기 식품명을 바로 확인할 수 있습니다.

🔍 메뉴 검색 기능
“김치”, “불고기”처럼 특정 메뉴를 입력하면 선택한 달 전체 급식에서 해당 메뉴가 나오는 날짜를 찾아줍니다.

🖥️ 위젯 기반 UI
DatePicker, Button, Text 등 **ipywidgets**를 사용해 코드를 몰라도 쉽게 사용할 수 있는 인터페이스를 제공합니다.


###  사용한 라이브러리
HTTP 요청을 보내기위해 requests라는 라이브러리를 사용했습니다.

날짜*시간처리를 위해 datetime 라이브러리를 사용했습니다.

UI*위젯 등을 구현하기 위해 각 widgets라이브러리,display라이브러리,HTML라이브러리,clear_output라이브러리를 추가했습니다.

특정 연*월의 마지막날짜를 구하기위해 monthrange 라이브러리를 사용했습니다.

나이스 교육정보 개방포털에서 받은 나이스API인증키를 보호하기 위해 코랩의 보안비밀 기능을 사용하였으며, userdata 라이브러리를 사용하였습니다.

### API 사용
저희는 `나이스 교육정보 개방포털`에서 나이스 API 인증키를 가져와 급식 데이터를 받았습니다.

### 학교 정보 설정
ATPT_OFCDC_SC_CODE 라는 변수에 부산광역시교육청코드인 "C10"을 대입했고
SD_SCHUL_CODE 라는 변수에 부산소마고 학교 코드인 7150658을 대입했습니다.

## 알레르기 번호 사전
급식 메뉴에 (1.2.6)처럼 표시된 숫자를 사람이 이해할수 있는 이름으로 바꾸기 위해 알레르기 번호 사전 코드를 딕셔너리 코드 형태로 작성하였습니다.

### 급식 데이터를 불러오는 함수
get_diet_data() == 특정날짜의 급식을 가져오는 함수

url 변수 == 나이스 급식 API주소

params = {} API 요청에 필요한 파라미터 묶음

"Key": API_KEY, # 인증키 선언

-----------------------------------------

"Type" : "json" # 응답형식을 JSON으로 받겠다는 뜻

-----------------------------------------

"pIndex" : 1, # 페이지 번호

"pSize": 10, # 한 페이지 데이터

-----------------------------------------

"ATPT_OFCDC_SC_CODE": ATPT_OFCDC_SC_CODE, #교육청코드

"SD_SCHUL_CODE": SD_SCHUL_CODE, # 학교코드

-----------------------------------------

## 다음에 기회가 된다면, 이 BSSM급식알리미를 웹사이트로 만들어보고싶습니다.

``` 발표자료 ```
 - 최종발표: https://www.canva.com/design/DAG-cr03iOA/Mv2eOjH8u8-w19jBBw8qVQ/edit?utm_content=DAG-cr03iOA&utm_campaign=designshare&utm_medium=link2&utm_source=sharebutton
 - 기획발표: https://www.canva.com/design/DAG-ct_wuNY/aaUWaDrOOvBS5mWR1u49Qw/edit?utm_content=DAG-ct_wuNY&utm_campaign=designshare&utm_medium=link2&utm_source=sharebutton

 ``` 노션 링크 ```
 https://www.notion.so/2e754c67e38080c8a9abde9d3d1a392e?source=copy_link



```Neis_APIKey```
5c2f9ee4dec94ff4a53bf00bb3c3df4e

In [ ]:
import requests
from datetime import datetime
import re
import ipywidgets as widgets
from IPython.display import display, HTML, clear_output
from calendar import monthrange
from google.colab import userdata

# 1. 나이스 API 인증키
API_KEY = userdata.get('Neis_APIKey')

# 2. 소마고 학교 정보
ATPT_OFCDC_SC_CODE = "C10"  # 부산시교육청
SD_SCHUL_CODE = "7150658"   # 부산소마고

# 3. 알레르기 번호
ALLERGY_DICT = {
    1: "난류",
    2: "우유",
    3: "메밀",
    4: "땅콩",
    5: "대두",
    6: "밀",
    7: "고등어",
    8: "게",
    9: "새우",
    10: "돼지고기",
    11: "복숭아",
    12: "토마토",
    13: "아황산류",
    14: "호두",
    15: "닭고기",
    16: "쇠고기",
    17: "오징어",
    18: "조개류(굴, 전복, 홍합 포함)"
}

# 4. 급식 정보 import
def get_diet_data(date_str):
    url = "https://open.neis.go.kr/hub/mealServiceDietInfo"

    params = {
        "KEY": API_KEY,
        "Type": "json",
        "pIndex": 1,
        "pSize": 10,
        "ATPT_OFCDC_SC_CODE": ATPT_OFCDC_SC_CODE,
        "SD_SCHUL_CODE": SD_SCHUL_CODE,
        "MLSV_YMD": date_str
    }

    try:
        response = requests.get(url, params=params) #나이스 서버로 GET 요청 전송
        data = response.json() # 응답을 JSON -> Python 딕셔너리로 변환

        if "mealServiceDietInfo" in data: # 정상적으로 급식 데이터가 있을경우
            return data["mealServiceDietInfo"][1]["row"] # 실제 급식 정보 목록만 반환
        elif "RESULT" in data:
            return data["RESULT"]["MESSAGE"] # 급식이 없을때 나오는 안내 메시지 반환
        else:
            return None
    except Exception as e:
        return f"error: {e}" # 예외 발생시 에러 메시지 반환

# 5. 알레르기 버튼 클릭시 이벤트 발생
def show_allergy_info(b):
    with output:
        print(f"⚠️ 알레르기 {b.allergy_num}번 : {ALLERGY_DICT.get(b.allergy_num, '정보 없음')}") # 버튼에 저장된 알레르기 번호를 가져와 해당 알레르기 이름 출력

# 6. 급식 확인 버튼 클릭시 이벤트 발생
def on_button_clicked(b):
    with output:
        clear_output() # 이전 출력 내용 제거

        selected_date = date_picker.value.strftime("%Y%m%d") # 선택한 날짜를 YYYYMMDD 형식으로 변환
        meals = get_diet_data(selected_date) # 해당 날짜의 급식 데이터 가져와 meals 변수에 저장

        print(f"📅 조회 날짜: {date_picker.value.strftime('%Y년 %m월 %d일')}\n")

        if meals is None:
            print("📭 해당 날짜의 급식 정보가 없습니다.")
            return

        if isinstance(meals, str):
            print(f"❌ 알림: {meals}")
            return

        for meal in meals: # 조식/중식/석식 반복
            print(f"🍴 [{meal['MMEAL_SC_NM']}]") # 매뉴 원본 문자열

            raw_menu = meal["DDISH_NM"]

            # 알레르기 번호 추출
            allergy_nums = re.findall(r'\d+', raw_menu)
            allergy_nums = sorted(set(map(int, allergy_nums)))

            # 메뉴에서 알레르기 표기 제거
            menu = re.sub(r'\(\d+(\.\d+)*\)', '', raw_menu)
            menu = menu.replace("<br/>", "\n   ")
            # 줄바꿈 처리

            print(f"   {menu}\n")

            # 알레르기 버튼 출력
            if allergy_nums:
                print("⚠️ 포함 알레르기 (클릭하세요)")
                btns = []

                for num in allergy_nums:
                    btn = widgets.Button(
                        description=str(num),
                        layout=widgets.Layout(width="45px"),
                        button_style="warning"
                    )
                    btn.allergy_num = num
                    btn.on_click(show_allergy_info)
                    btns.append(btn)

                display(widgets.HBox(btns))

            print("-" * 50)

# 선택한 달의 모든 날짜 생성(ver2 추가)
def get_month_dates(date):
    year = date.year
    month = date.month
    last_day = monthrange(year, month)[1]
    return [f"{year}{month:02d}{day:02d}" for day in range(1, last_day + 1)]

# 메뉴 검색 버튼 클릭시 이벤트 발생
def on_menu_search_clicked(b):
    with output:
        clear_output()

        keyword = menu_search.value.strip()
        if not keyword:
            print("❗ 검색할 메뉴를 입력하세요.")
            return

        print(f"🔍 검색 메뉴: {keyword}\n")

        dates = get_month_dates(date_picker.value)
        found = False

        for date_str in dates:
            meals = get_diet_data(date_str)
            if not isinstance(meals, list):
                continue

            for meal in meals:
                if keyword in meal["DDISH_NM"]:
                    found = True
                    date_fmt = datetime.strptime(date_str, "%Y%m%d").strftime("%Y년 %m월 %d일")

                    menu = re.sub(r'\(\d+(\.\d+)*\)', '', meal["DDISH_NM"])
                    menu = menu.replace("<br/>", "\n   ")

                    print(f"📅 {date_fmt}")
                    print(f"🍴 [{meal['MMEAL_SC_NM']}]")
                    print(f"   {menu}")
                    print("-" * 50)

        if not found:
            print("❌ 해당 메뉴가 포함된 급식을 찾지 못했습니다.")

# 7. 위젯 UI 구성
date_picker = widgets.DatePicker( # 날짜 선택기
    description="날짜 선택:",
    value=datetime.now().date()
)

button = widgets.Button( # 버튼
    description="급식 불러오기",
    button_style="success",
    icon="search"
)

menu_search = widgets.Text( # 텍스트 입력창
    description="메뉴 검색:",
    placeholder="검색할 메뉴를 입력하세요."
)

search_button = widgets.Button(
    description="메뉴 검색",
    button_style="info",
    icon="search"
)

button.on_click(on_button_clicked) # 이벤트 연결
search_button.on_click(on_menu_search_clicked) # 이벤트 연결

output = widgets.Output()

# 8. 화면 출력
display(HTML("<h2>🏫 BSSM 급식 알리미</h2>"))
display(widgets.HBox([date_picker, button])) # 날짜 선택
display(widgets.HBox([menu_search, search_button])) # 메뉴검색
display(output)
